# Adding images for training from a video

This notebook is similar to the previous one but the source of the images to label will be a video.

As an example here, we will generate the video from a camera and then work from the video.


You eventually want to have at least 500 images in your dataset. You can start with 100-200 images and go through all notebooks. You can always add more images and re-train your network. 

You probably want a minimum of 150 images to get started.

In [16]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import ipywidgets as widgets
from ipyevents import Event 
import threading
from IPython.display import display


from unetTracker.trackingProject import TrackingProject
from unetTracker.dataset import UNetDataset
from unetTracker.camera import USBCamera, bgr8_to_jpeg
from unetTracker.unetGUI import LabelFromImagesGUI

Load a project

In [17]:
project = TrackingProject(name="mouseTrack",root_folder = "/home/kevin/Documents/trackingProjects/")

Project directory: /home/kevin/Documents/trackingProjects/mouseTrack
Loading /home/kevin/Documents/trackingProjects/mouseTrack/config.yalm
{'augmentation_HorizontalFlipProb': 0.0, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_size': [480, 480], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'mouseTrack', 'normalization_values': {'means': [0.4181702136993408, 0.4182196259498596, 0.41791409254074097], 'stds': [0.11457645893096924, 0.11454392969608307, 0.11477964371442795]}, 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (240.0, 255.0, 255.0)], 'objects': ['snout', 'earL', 'earR', 'tail'], 'target_radius': 5}


Create a dataset for the project.

In [18]:
dataset = UNetDataset(image_dir=project.image_dir, mask_dir=project.mask_dir, coordinate_dir=project.coordinate_dir)

In [19]:
print("Number of images in the dataset:",len(dataset))

Number of images in the dataset: 50


### Creating a video as an example

In this notebook, we will label images from a video. 

In case you don't have a video, you can use a camera to create a small video to work with.

**If you already have your video, jump directly to "Extract Frames from a Video" section**

In [5]:
#!ls -ltrh /dev/video*

Get a camera to create a video

In [6]:
#camera = USBCamera(width=project.image_size[1], height=project.image_size[0])

In [7]:
#frame = camera.read()
#plt.imshow(frame)

Save a video from the camera.

In [8]:
#frame2save=30*10
#video_fn = '/tmp/video1.avi'
#size=project.image_size[1],project.image_size[0]
#video = cv2.VideoWriter(video_fn, cv2.VideoWriter_fourcc(*'MJPG'),30, size)

# loop to save the video
#for i in range(frame2save):
#    frame = camera.read()
#    video.write(frame)
    
#video.release()

## Extract frames from a video

You need to select a directory in which the individual extracted frames will be saved. Here I used a directory within my project directory.

Images will be added to any image that is already in the folder. 

In [20]:
video_fn = "/ext_drives/d69/data/electro/fjk9263/fjk9263-17112022-1221/output.mp4"
extracted_frame_dir = project.project_dir+"/extracted_frames/"
print(extracted_frame_dir)

/home/kevin/Documents/trackingProjects/mouseTrack/extracted_frames/


In [21]:
dataset.extract_frames_from_video(video_fn,50,extracted_frame_dir)

Extracting frames: [   2   49  214  250  260  542  670  988 1032 1395 1573 1605 1629 1639
 1786 1813 2135 2184 2366 2469 2534 2694 2763 2851 3120 3209 3225 3338
 3439 3832 3872 4170 4277 4320 4487 4516 4699 4706 4856 4917 4976 5136
 5630 5632 5664 5734 5887 6333 6591 7104] to /home/kevin/Documents/trackingProjects/mouseTrack/extracted_frames/


## Label extracted frames and save to dataset

We use a GUI to label the object in the extracted frames.

Make sure that your image is shown at maximal size by extending the notebook window. 
Make sure the label are correctly positioned in the image below.


1. In the large image, click on the object selected by the radio button. The label should appear in the picture below. 
2. If you don't want to save the data from a particular image, click on Next frame.
2. Repeat for all your visible objects
3. Click on Save labelled frame.
4. Repeat for all your images

When you click on `Save labelled frame`, the image is remove from the `extract_frame_dir` directory and transfer to your dataset.

In [22]:
project.labeling_ImageEnlargeFactor = 2.0

In [23]:
LabelFromImagesGUI(image_dir=extracted_frame_dir,project=project,dataset=dataset)

In [24]:
len(dataset)

100